### Imports

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

import time

### Load Data

In [2]:
DATA_PATH = "../../data/Forest-Cover/"

train = pd.read_csv(DATA_PATH + "train.csv")
test  = pd.read_csv(DATA_PATH + "test.csv")

cat_columns = [col for col in train.columns if train[col].dtype == object]
print("Categorical columns:")
print(" --- ".join(cat_columns))

### Numerical columns
num_columns = [col for col in train.columns if train[col].dtype != object]
print("Numerical columns:")
print(" --- ".join(num_columns))
print()
print("Shape of train:", train.shape)
print("Shape of test:",  test.shape)

Categorical columns:

Numerical columns:
Id --- Elevation --- Aspect --- Slope --- Horizontal_Distance_To_Hydrology --- Vertical_Distance_To_Hydrology --- Horizontal_Distance_To_Roadways --- Hillshade_9am --- Hillshade_Noon --- Hillshade_3pm --- Horizontal_Distance_To_Fire_Points --- Wilderness_Area1 --- Wilderness_Area2 --- Wilderness_Area3 --- Wilderness_Area4 --- Soil_Type1 --- Soil_Type2 --- Soil_Type3 --- Soil_Type4 --- Soil_Type5 --- Soil_Type6 --- Soil_Type7 --- Soil_Type8 --- Soil_Type9 --- Soil_Type10 --- Soil_Type11 --- Soil_Type12 --- Soil_Type13 --- Soil_Type14 --- Soil_Type15 --- Soil_Type16 --- Soil_Type17 --- Soil_Type18 --- Soil_Type19 --- Soil_Type20 --- Soil_Type21 --- Soil_Type22 --- Soil_Type23 --- Soil_Type24 --- Soil_Type25 --- Soil_Type26 --- Soil_Type27 --- Soil_Type28 --- Soil_Type29 --- Soil_Type30 --- Soil_Type31 --- Soil_Type32 --- Soil_Type33 --- Soil_Type34 --- Soil_Type35 --- Soil_Type36 --- Soil_Type37 --- Soil_Type38 --- Soil_Type39 --- Soil_Type40 --- 

### Split into id, target, and predictors

In [3]:
train_y = train["Cover_Type"]
train_id = train["Id"]
train_x = train.drop(["Cover_Type", "Id"], axis=1)

test_id = test["Id"]
test_x  = test.drop("Id", axis=1)

full    = pd.concat([train_x, test_x])
train_N = len(train_x)

### Create Features

In [4]:
full['HF1'] = abs(full['Horizontal_Distance_To_Hydrology']   + full['Horizontal_Distance_To_Fire_Points'])
full['HF2'] = abs(full['Horizontal_Distance_To_Hydrology']   - full['Horizontal_Distance_To_Fire_Points'])
full['HR1'] = abs(full['Horizontal_Distance_To_Hydrology']   + full['Horizontal_Distance_To_Roadways'])
full['HR2'] = abs(full['Horizontal_Distance_To_Hydrology']   - full['Horizontal_Distance_To_Roadways'])
full['FR1'] = abs(full['Horizontal_Distance_To_Fire_Points'] + full['Horizontal_Distance_To_Roadways'])
full['FR2'] = abs(full['Horizontal_Distance_To_Fire_Points'] - full['Horizontal_Distance_To_Roadways'])
full['ele_vert'] = full.Elevation-full.Vertical_Distance_To_Hydrology

full['slope_hyd'] = (full['Horizontal_Distance_To_Hydrology']**2 + full['Vertical_Distance_To_Hydrology']**2)**0.5
full["slope_hyd"] = full.slope_hyd.map(lambda x: 0 if np.isinf(x) else x)

full['Mean_Amenities'] = (full.Horizontal_Distance_To_Fire_Points +
                          full.Horizontal_Distance_To_Hydrology +
                          full.Horizontal_Distance_To_Roadways) / 3 
full['Mean_Fire_Hyd']  = (full.Horizontal_Distance_To_Fire_Points +
                          full.Horizontal_Distance_To_Hydrology) / 2 

### Split back into train and test

In [5]:
train_x = full[:train_N]
test_x  = full[train_N:]

### Machine Learning Imports

In [6]:
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier,
                              ExtraTreesClassifier, VotingClassifier)
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

TAKE_CV = False
first_layer_train = pd.DataFrame()
first_layer_preds = pd.DataFrame()

### Extra Trees 1

This works really well, so I'm going to create a second model with different params and random_state

In [8]:
etc_model_1 = ExtraTreesClassifier(n_estimators=25, criterion='gini',
                                   max_depth=None, min_samples_split=2,
                                   min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                                   max_features='auto', max_leaf_nodes=None,
                                   min_impurity_decrease=0.0, min_impurity_split=None,
                                   bootstrap=False, random_state=17)

if TAKE_CV:
    scores = cross_val_score(etc_model_1, train_x, train_y, cv=5, verbose=1)
    score_mean = round(np.mean(scores), 4)
    score_std  = round(np.std(scores), 3)
    print(f"Score is {score_mean} +/- {score_std}")

etc_model_1.fit(train_x, train_y)

first_layer_train["ETC1"] = etc_model_1.predict(train_x)
first_layer_preds["ETC1"] = etc_model_1.predict(test_x);

### Extra Trees 2

In [9]:
etc_model_2 = ExtraTreesClassifier(n_estimators=500, criterion='gini',
                                   max_depth=None, min_samples_split=2,
                                   min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                                   max_features='auto', max_leaf_nodes=None,
                                   min_impurity_decrease=0.0, min_impurity_split=None,
                                   bootstrap=False, random_state=71)

if TAKE_CV:
    scores = cross_val_score(etc_model_2, train_x, train_y, cv=5, verbose=1)
    score_mean = round(np.mean(scores), 4)
    score_std  = round(np.std(scores), 3)
    print(f"Score is {score_mean} +/- {score_std}")

etc_model_2.fit(train_x, train_y)

first_layer_train["ETC2"] = etc_model_2.predict(train_x)
first_layer_preds["ETC2"] = etc_model_2.predict(test_x);

### XGB Model 1

In [10]:
xgb_model_1 = XGBClassifier(max_depth=4, learning_rate=1,
                            n_estimators=100, gamma=0, min_child_weight=1,
                            max_delta_step=0, subsample=1, colsample_bytree=0.6,
                            colsample_bylevel=1, reg_alpha=3, reg_lambda=3,
                            scale_pos_weight=1, base_score=0.5, random_state=17)

if TAKE_CV:
    scores = cross_val_score(xgb_model_1, train_x, train_y, cv=5, verbose=1)
    score_mean = round(np.mean(scores), 4)
    score_std  = round(np.std(scores), 3)
    print(f"Score is {score_mean} +/- {score_std}")

xgb_model_1.fit(train_x, train_y)

first_layer_train["XGB1"] = xgb_model_1.predict(train_x)
first_layer_preds["XGB1"] = xgb_model_1.predict(test_x);

### XGB Model 2

This model is taken from [Siddharth Yadav](https://www.kaggle.com/thebrownviking20) and his excellent [kernel](https://www.kaggle.com/thebrownviking20/voting-classifier-for-victory)

In [ ]:
xgb_model_2 = XGBClassifier(max_depth=20, n_estimators=1000, random_state=71)

if TAKE_CV:
    scores = cross_val_score(xgb_model_2, train_x, train_y, cv=5, verbose=1)
    score_mean = round(np.mean(scores), 4)
    score_std  = round(np.std(scores), 3)
    print(f"Score is {score_mean} +/- {score_std}")

xgb_model_2.fit(train_x, train_y)

first_layer_train["XGB2"] = xgb_model_2.predict(train_x)
first_layer_preds["XGB2"] = xgb_model_2.predict(test_x);

### Random Forest

In [ ]:
rfc_model = RandomForestClassifier(n_estimators=25, criterion='gini',
                                   max_depth=None, min_samples_split=6,
                                   min_samples_leaf=3, min_weight_fraction_leaf=0.0,
                                   max_features='auto', max_leaf_nodes=None,
                                   min_impurity_decrease=0.0, min_impurity_split=None,
                                   random_state=17)

if TAKE_CV:
    scores = cross_val_score(rfc_model, train_x, train_y, cv=5, verbose=1)
    score_mean = round(np.mean(scores), 4)
    score_std  = round(np.std(scores), 3)
    print(f"Score is {score_mean} +/- {score_std}")

rfc_model.fit(train_x, train_y)

first_layer_train["RFC"] = rfc_model.predict(train_x)
first_layer_preds["RFC"] = rfc_model.predict(test_x);

### Light Gradient Boosting 1

In [ ]:
lgb_model_1  = LGBMClassifier(num_leaves=45, max_depth=7,
                              learning_rate=0.3,
                              reg_lambda=0.5, reg_alpha=0.5,
                              min_split_gain=0.1, min_child_weight=0.5,
                              min_data_in_leaf=5,
                              feature_fraction=0.5,
                              random_state=17)

if TAKE_CV:
    scores = cross_val_score(lgb_model_1, train_x, train_y, cv=5, verbose=1)
    score_mean = round(np.mean(scores), 4)
    score_std  = round(np.std(scores), 3)
    print(f"Score is {score_mean} +/- {score_std}")

lgb_model_1.fit(train_x, train_y)

first_layer_train["LGB1"] = lgb_model_1.predict(train_x)
first_layer_preds["LGB1"] = lgb_model_1.predict(test_x);

### Light Gradient Boosting 2

In [ ]:
lgb_model_2  = LGBMClassifier(num_leaves=70, max_depth=8,
                              learning_rate=0.1,
                              reg_lambda=1, reg_alpha=1,
                              min_split_gain=0.1, min_child_weight=0.5,
                              min_data_in_leaf=5,
                              feature_fraction=0.3,
                              random_state=71)

if TAKE_CV:
    scores = cross_val_score(lgb_model_2, train_x, train_y, cv=5, verbose=1)
    score_mean = round(np.mean(scores), 4)
    score_std  = round(np.std(scores), 3)
    print(f"Score is {score_mean} +/- {score_std}")

lgb_model_2.fit(train_x, train_y)

first_layer_train["LGB2"] = lgb_model_2.predict(train_x)
first_layer_preds["LGB2"] = lgb_model_2.predict(test_x);

### Light Gradient Boosting 3

This model is taken from [Siddharth Yadav](https://www.kaggle.com/thebrownviking20) and his excellent [kernel](https://www.kaggle.com/thebrownviking20/voting-classifier-for-victory)

In [ ]:
lgb_model_3  = LGBMClassifier(n_estimators=2000, max_depth=15, random_state=171)

if TAKE_CV:
    scores = cross_val_score(lgb_model_3, train_x, train_y, cv=5, verbose=1)
    score_mean = round(np.mean(scores), 4)
    score_std  = round(np.std(scores), 3)
    print(f"Score is {score_mean} +/- {score_std}")

lgb_model_3.fit(train_x, train_y)

first_layer_train["LGB3"] = lgb_model_3.predict(train_x)
first_layer_preds["LGB3"] = lgb_model_3.predict(test_x);

### Ada Boost 1

This model is taken from [Siddharth Yadav](https://www.kaggle.com/thebrownviking20) and his excellent [kernel](https://www.kaggle.com/thebrownviking20/voting-classifier-for-victory)

In [ ]:
ada_model_1 = AdaBoostClassifier(
                ExtraTreesClassifier(n_estimators=500),
                n_estimators=250, learning_rate=0.01, algorithm='SAMME', random_state=17
)

if TAKE_CV:
    scores = cross_val_score(ada_model_1, train_x, train_y, cv=5, verbose=1)
    score_mean = round(np.mean(scores), 4)
    score_std  = round(np.std(scores), 3)
    print(f"Score is {score_mean} +/- {score_std}")

ada_model_1.fit(train_x, train_y)

first_layer_train["ADA1"] = ada_model_1.predict(train_x)
first_layer_preds["ADA1"] = ada_model_1.predict(test_x);

### Ada Boost 2

This model is taken from [Siddharth Yadav](https://www.kaggle.com/thebrownviking20) and his excellent [kernel](https://www.kaggle.com/thebrownviking20/voting-classifier-for-victory)

In [ ]:
ada_model_2 = AdaBoostClassifier(
                GradientBoostingClassifier(n_estimators=1000, max_depth=10),
                n_estimators=1000, learning_rate=0.01, algorithm="SAMME", random_state=17
)

if TAKE_CV:
    scores = cross_val_score(ada_model_2, train_x, train_y, cv=5, verbose=1)
    score_mean = round(np.mean(scores), 4)
    score_std  = round(np.std(scores), 3)
    print(f"Score is {score_mean} +/- {score_std}")

ada_model_2.fit(train_x, train_y)

first_layer_train["ADA2"] = ada_model_2.predict(train_x)
first_layer_preds["ADA2"] = ada_model_2.predict(test_x);

### Ensemble Classifer

In [ ]:
voting_model = SVC()

if TAKE_CV:
    scores = cross_val_score(voting_model, first_layer_train, train_y, cv=5, verbose=1)
    score_mean = round(np.mean(scores), 4)
    score_std  = round(np.std(scores), 3)
    print(f"Score is {score_mean} +/- {score_std}")
    
voting_model.fit(first_layer_train, train_y)
predictions = voting_model.predict(first_layer_preds)

### Save Predictions

In [ ]:
predictions = pd.DataFrame({"Cover_Type": predictions, "Id": test_id} )
predictions.to_csv("../", index=False)